# 深層学習体験会: 画像から動画生成（開始・終了フレーム指定）

このノートブックでは、**LTX-Video**モデルを使って、2枚の画像から滑らかな動画を生成する方法を学びます。

## LTX-Videoとは？

**LTX-Video**は、画像とテキストプロンプトから動画を生成する最新のAIモデルです。このノートブックでは、特に**2枚の画像間を補間する**機能を使用します。

### 主な機能
- **開始・終了フレーム指定**: 2枚の画像の間を滑らかに補間
- **プロンプト制御**: テキストで動画の内容を指示
- **高品質出力**: 滑らかで自然な動画生成

### 使用例
- 静止画のアニメーション化
- 商品画像の動的プレゼンテーション
- キャラクターのポーズ変化アニメーション
- 風景写真の時間変化表現

## 重要な注意事項
- 無料のT4 GPUで実行可能ですが、より高速な生成には上位GPUを推奨
- このノートブックは**シンプルな遷移**（例: 花が成長する、ポーズが変わる）に適しています
- 激しい動き（歩く、走る）には適していません
- **詳細なプロンプト**を使用すると、より良い結果が得られます

## このノートブックの流れ
1. 環境の準備とモデルのダウンロード
2. 2枚の画像をアップロード
3. プロンプトとパラメータの設定
4. 動画の生成と再生

In [ ]:
# --- 環境の準備 ---

# PyTorchとtorchvisionの特定バージョンをインストール（LTX-Video用）
!pip install torch==2.6.0 torchvision==0.21.0

# /contentディレクトリに移動
%cd /content

# モデル読み込みの設定
Always_Load_Models_for_Inference = False  # 常にモデルを推論用に読み込むか
Use_t5xxl_fp16 = False  # 16bit精度のテキストエンコーダーを使用するか（Falseで8bit軽量版を使用）

# 必要なライブラリをインストール
# -q: 静かにインストール（詳細なログを表示しない）
!pip install -q torchsde einops diffusers accelerate xformers==0.0.29.post2
!pip install av  # 動画ファイル処理用

# ComfyUIをクローン（動画生成のバックエンド）
!git clone https://github.com/Isi-dev/ComfyUI
%cd /content/ComfyUI

# aria2（高速ダウンローダー）とffmpeg（動画処理）をインストール
!apt -y install -qq aria2 ffmpeg

# --- モデルファイルのダウンロード ---
# メインの動画生成モデルをダウンロード
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Isi99999/LTX-Video/resolve/main/ltx-video-2b-v0.9.5.safetensors -d /content/ComfyUI/models/checkpoints -o ltx-video-2b-v0.9.5.safetensors

# テキストエンコーダーのダウンロード（設定に応じて16bitまたは8bit版）
if Use_t5xxl_fp16:
    # 16bit版（高精度だが重い）
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Isi99999/LTX-Video/resolve/main/t5xxl_fp16.safetensors -d /content/ComfyUI/models/text_encoders -o t5xxl_fp16.safetensors
else:
    # 8bit版（軽量で推奨）
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Isi99999/LTX-Video/resolve/main/t5xxl_fp8_e4m3fn_scaled.safetensors -d /content/ComfyUI/models/text_encoders -o t5xxl_fp8_e4m3fn_scaled.safetensors

# --- 必要なライブラリをインポート ---
import torch  # PyTorch（ディープラーニングフレームワーク）
import numpy as np  # 数値計算ライブラリ
from PIL import Image  # 画像処理
import gc  # ガベージコレクション（メモリ管理）
import sys  # システム関連
import random  # 乱数生成
import os  # OS操作
import imageio  # 動画ファイルの読み書き
from google.colab import files  # Google Colabでのファイルアップロード
from IPython.display import display, HTML  # Jupyter Notebookでの表示

# ComfyUIのパスをシステムパスに追加
sys.path.insert(0, '/content/ComfyUI')

# ComfyUIのモジュールをインポート
from comfy import model_management

# 基本的なノード（処理単位）をインポート
from nodes import (
    CheckpointLoaderSimple,  # モデル読み込み
    CLIPLoader,  # テキストエンコーダー読み込み
    CLIPTextEncode,  # テキストのエンコード
    VAEDecode,  # 潜在表現から画像へのデコード
    LoadImage  # 画像読み込み
)

# カスタムサンプラー関連のノード
from comfy_extras.nodes_custom_sampler import (
    KSamplerSelect,  # サンプラーの選択
    SamplerCustom  # カスタムサンプラー
)

# LTX-Video専用のノード
from comfy_extras.nodes_lt import (
    EmptyLTXVLatentVideo,  # 空の潜在動画を作成
    LTXVPreprocess,  # 画像の前処理
    LTXVAddGuide,  # ガイド画像の追加
    LTXVScheduler,  # ノイズスケジュール
    LTXVConditioning,  # 条件付け
    LTXVCropGuides  # ガイドのクロップ
)

# --- ノードのインスタンスを作成 ---
checkpoint_loader = CheckpointLoaderSimple()  # モデル読み込み用
clip_loader = CLIPLoader()  # CLIP（テキストエンコーダー）読み込み用
clip_encode_positive = CLIPTextEncode()  # ポジティブプロンプトのエンコード用
clip_encode_negative = CLIPTextEncode()  # ネガティブプロンプトのエンコード用
load_image = LoadImage()  # 画像読み込み用
empty_latent = EmptyLTXVLatentVideo()  # 空の潜在動画作成用
preprocess = LTXVPreprocess()  # 画像前処理用
add_guide = LTXVAddGuide()  # ガイド画像追加用
scheduler = LTXVScheduler()  # スケジューラー用
sampler_select = KSamplerSelect()  # サンプラー選択用
conditioning = LTXVConditioning()  # 条件付け用
sampler = SamplerCustom()  # サンプリング実行用
vae_decode = VAEDecode()  # デコード用
crop_guides = LTXVCropGuides()  # ガイドクロップ用

def clear_gpu_memory():
    """
    GPUメモリを解放する関数
    動画生成後にメモリをクリーンアップするために使用
    """
    # Pythonのガベージコレクションを実行
    gc.collect()
    
    # CUDAが利用可能な場合
    if torch.cuda.is_available():
        # キャッシュをクリア
        torch.cuda.empty_cache()
        # プロセス間通信のキャッシュもクリア
        torch.cuda.ipc_collect()
    
    # グローバル変数からテンソルを削除
    for obj in list(globals().values()):
        if torch.is_tensor(obj) or (hasattr(obj, "data") and torch.is_tensor(obj.data)):
            del obj
    
    # 再度ガベージコレクション
    gc.collect()

def upload_image():
    """
    Google Colabで画像をアップロードし、ComfyUIのinputディレクトリに保存する関数
    
    戻り値:
        str: 保存された画像のパス（失敗時はNone）
    """
    from google.colab import files
    import os
    import shutil

    # inputディレクトリを作成（既に存在する場合はエラーにしない）
    os.makedirs('/content/ComfyUI/input', exist_ok=True)

    # ファイルアップロードのUIを表示
    uploaded = files.upload()

    # アップロードされた各ファイルを処理
    for filename in uploaded.keys():
        # 元のパスと保存先のパス
        src_path = f'/content/ComfyUI/{filename}'
        dest_path = f'/content/ComfyUI/input/{filename}'

        # ファイルを移動
        shutil.move(src_path, dest_path)
        print(f"画像を保存しました: {dest_path}")
        return dest_path

    return None

def generate_video(
    image_path: str = None,  # 開始画像のパス
    guide_image_path: str = None,  # 終了画像のパス
    positive_prompt: str = "A red fox moving gracefully, its russet coat vibrant against the white landscape, leaving perfect star-shaped prints behind as steam rises from its breath in the crisp winter air. The scene is wrapped in snow-muffled silence, broken only by the gentle murmur of water still flowing beneath the ice.",  # ポジティブプロンプト
    negative_prompt: str = "low quality, worst quality, deformed, distorted, disfigured, motion smear, motion artifacts, fused fingers, bad anatomy, weird hand, ugly",  # ネガティブプロンプト
    width: int = 768,  # 動画の幅
    height: int = 512,  # 動画の高さ
    seed: int = 397166166231987,  # ランダムシード
    steps: int = 30,  # 生成ステップ数
    cfg_scale: float = 2.05,  # CFGスケール（プロンプトへの忠実度）
    sampler_name: str = "euler",  # サンプラー名
    length: int = 97,  # フレーム数
    fps: int = 24,  # フレームレート
    guide_strength: float = 0.1,  # ガイド強度
    guide_frame: int = -1  # ガイドを適用するフレーム位置（-1で最後）
):
    """
    2枚の画像から動画を生成するメイン関数
    
    引数:
        image_path: 開始画像のパス
        guide_image_path: 終了画像のパス
        positive_prompt: 生成したい内容の説明
        negative_prompt: 避けたい要素の説明
        width: 動画の幅（32の倍数）
        height: 動画の高さ（32の倍数）
        seed: ランダムシード（再現性のため）
        steps: 生成ステップ数（多いほど高品質）
        cfg_scale: プロンプトへの忠実度
        sampler_name: サンプリング方法
        length: フレーム数
        fps: フレームレート
        guide_strength: 終了画像への影響度
        guide_frame: 終了画像を配置する位置
    """
    # 推論モード（学習しない）で実行
    with torch.inference_mode():
        print("テキストエンコーダーを読み込み中...")
        # CLIPテキストエンコーダーを読み込む
        clip = clip_loader.load_clip("t5xxl_fp8_e4m3fn_scaled.safetensors", "ltxv", "default")[0]
        print("テキストエンコーダーを読み込みました！")

    try:
        # --- 解像度の検証 ---
        # 幅と高さが32の倍数であることを確認
        assert width % 32 == 0, "幅は32の倍数である必要があります"
        assert height % 32 == 0, "高さは32の倍数である必要があります"

        # --- プロンプトのエンコード ---
        # ポジティブプロンプト（生成したい内容）をエンコード
        positive = clip_encode_positive.encode(clip, positive_prompt)[0]
        # ネガティブプロンプト（避けたい内容）をエンコード
        negative = clip_encode_negative.encode(clip, negative_prompt)[0]

        # テキストエンコーダーを削除してメモリを解放
        del clip
        torch.cuda.empty_cache()
        gc.collect()
        print("テキストエンコーダーをメモリから削除しました")

        # --- 画像のアップロード ---
        # 開始画像がない場合はアップロードを促す
        if image_path is None:
            print("開始画像ファイルをアップロードしてください:")
            image_path = upload_image()
        if image_path is None:
            print("開始画像がアップロードされていません！")

        # 終了画像がない場合はアップロードを促す
        if guide_image_path is None:
            print("終了画像ファイルをアップロードしてください:")
            guide_image_path = upload_image()
        if guide_image_path is None:
            print("終了画像がアップロードされていません！")

        # --- 画像の読み込みと前処理 ---
        # 開始画像を読み込む
        loaded_image = load_image.load_image(image_path)[0]
        # 前処理を実行（リサイズなど）
        processed_image = preprocess.preprocess(loaded_image, 35)[0]

        # 終了画像を読み込む
        loaded_guide_image = load_image.load_image(guide_image_path)[0]
        # 前処理を実行
        processed_guide_image = preprocess.preprocess(loaded_guide_image, 40)[0]

        # --- モデルとVAEの読み込み ---
        print("モデルとVAEを読み込み中...")
        # チェックポイントからモデル、CLIP、VAEを読み込む
        model, _, vae = checkpoint_loader.load_checkpoint("ltx-video-2b-v0.9.5.safetensors")
        print("モデルとVAEを読み込みました！")

        # --- 空の潜在動画を作成 ---
        # 指定された幅、高さ、長さで空の潜在表現を作成
        latent_video = empty_latent.generate(width, height, length)[0]

        # --- 1回目のガイド追加（開始画像） ---
        # 最初のフレーム（frame_idx=0）に開始画像を追加
        # strength=1: 完全にガイド画像に従う
        guided_positive, guided_negative, guided_latent_1 = add_guide.generate(
            positive=positive,
            negative=negative,
            vae=vae,
            latent=latent_video,
            image=processed_image,
            frame_idx=0,  # 最初のフレーム
            strength=1  # 強度100%
        )

        # --- 2回目のガイド追加（終了画像） ---
        # 指定されたフレーム位置に終了画像を追加
        guided_positive, guided_negative, guided_latent = add_guide.generate(
            positive=guided_positive,
            negative=guided_negative,
            vae=vae,
            latent=guided_latent_1,
            image=processed_guide_image,
            frame_idx=guide_frame,  # 終了フレーム（-1で最後）
            strength=guide_strength  # ガイド強度（0〜1）
        )

        # --- サンプリングの準備 ---
        # ノイズスケジュールを取得（生成プロセスのステップを制御）
        sigmas = scheduler.get_sigmas(steps, cfg_scale, 0.95, True, 0.1, guided_latent_1)[0]
        # サンプラーを選択（eulerなど）
        selected_sampler = sampler_select.get_sampler(sampler_name)[0]

        # --- 条件付けの適用 ---
        # プロンプトの条件付けを適用
        conditioned_positive, conditioned_negative = conditioning.append(
            guided_positive,
            guided_negative,
            25.0  # 条件付けの強度
        )

        print("動画を生成中...")

        # --- 動画のサンプリング ---
        # メインの生成プロセス
        sampled = sampler.sample(
            model=model,  # 使用するモデル
            add_noise=True,  # ノイズを追加
            noise_seed=seed if seed != 0 else random.randint(0, 2**32),  # シード
            cfg=cfg_scale,  # CFGスケール
            positive=conditioned_positive,  # ポジティブ条件
            negative=conditioned_negative,  # ネガティブ条件
            sampler=selected_sampler,  # サンプラー
            sigmas=sigmas,  # ノイズスケジュール
            latent_image=guided_latent  # 入力潜在表現
        )[0]

        # --- ガイドのクロップ ---
        # ガイドフレームをクロップ（必要に応じて）
        cropped_latent = crop_guides.crop(
            conditioned_positive,
            conditioned_negative,
            sampled
        )[2]

        # モデルを削除してメモリを解放
        del model
        torch.cuda.empty_cache()
        gc.collect()
        print("モデルをメモリから削除しました")

        # --- 潜在表現のデコード ---
        # 勾配計算なしで実行
        with torch.no_grad():
            try:
                print("潜在表現をデコード中...")
                # VAEで潜在表現を動画フレームにデコード
                decoded = vae_decode.decode(vae, cropped_latent)[0].detach()
                print("潜在表現をデコードしました！")
                
                # VAEを削除してメモリを解放
                del vae
                torch.cuda.empty_cache()
                gc.collect()
                print("VAEをメモリから削除しました")
            except Exception as e:
                print(f"デコード中にエラーが発生しました: {str(e)}")
                raise

        # --- MP4ファイルとして保存 ---
        output_path = "/content/output.mp4"
        # フレームを0-255の範囲のuint8に変換
        frames_np = (decoded.cpu().numpy() * 255).astype(np.uint8)
        
        # imageioで動画ファイルを書き込む
        with imageio.get_writer(output_path, fps=fps) as writer:
            for frame in frames_np:
                writer.append_data(frame)

        # 完了メッセージ
        print(f"\n動画生成が完了しました！")
        print(f"{len(decoded)}フレームを{output_path}に保存しました")
        
        # 動画を表示
        display_video(output_path)

    except Exception as e:
        print(f"動画生成中にエラーが発生しました: {str(e)}")
        raise
    finally:
        # 最後に必ずメモリをクリーンアップ
        clear_gpu_memory()

def display_video(video_path):
    """
    Colab Notebookで動画を表示する関数
    HTML5プレーヤーを使用して動画を埋め込み表示
    
    引数:
        video_path: 動画ファイルのパス
    """
    from IPython.display import HTML
    from base64 import b64encode

    # 動画ファイルを読み込む
    mp4 = open(video_path,'rb').read()
    # Base64エンコードしてdata URLを作成
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

    # HTML5のvideoタグで表示
    display(HTML(f"""
    <video width=512 controls autoplay loop>
        <source src="{data_url}" type="video/mp4">
    </video>
    """))

## ステップ1: 環境の準備

このセルでは、以下の作業を行います:

### インストール
1. **PyTorch 2.6.0**: 深層学習フレームワーク
2. **Diffusers**: 拡散モデルライブラリ
3. **Accelerate**: モデルの高速化
4. **ComfyUI**: 動画生成のバックエンド

### モデルのダウンロード
- **ltx-video-2b-v0.9.5.safetensors**: メインの動画生成モデル
- **t5xxl_fp8_e4m3fn_scaled.safetensors**: テキストエンコーダー（軽量版）

### 関数の定義
- `generate_video()`: 動画生成のメイン関数
- `upload_image()`: 画像アップロード機能
- `display_video()`: 動画表示機能

実行には5〜10分かかる場合があります。「Environment Setup Complete!」と表示されれば準備完了です。

## ステップ2: 動画生成の実行

### 調整可能なパラメータ

#### プロンプト設定
- **positive_prompt**: 生成したい動画の内容を詳細に記述
  - 例: "Flowers growing from the sides of a vase"（花瓶の側面から花が成長する）
- **negative_prompt**: 避けたい要素を指定
  - 例: 低品質、歪み、動きのブレなど

#### 解像度設定
- **width**: 動画の幅（32の倍数である必要があります）
- **height**: 動画の高さ（32の倍数である必要があります）
- 推奨: 512x768 または 768x512

#### 生成パラメータ
- **seed**: ランダムシード（同じ値で同じ結果を再現）
- **steps**: 生成ステップ数（多いほど高品質だが時間がかかる）
- **cfg_scale**: プロンプトへの忠実度（2.0前後を推奨）
- **sampler_name**: サンプリング方法（euler推奨）

#### 動画設定
- **frames**: 生成するフレーム数（多いほど長い動画）
- **guide_strength**: 終了フレームへの影響度（0〜1、1が最大）
- **guide_frame**: 終了画像を配置するフレーム位置（-1で最後）

### 実行方法
1. セルを実行すると、最初に**開始画像**をアップロードするよう求められます
2. 次に**終了画像**をアップロードするよう求められます
3. 設定したパラメータで動画が生成されます
4. 完了すると、動画が自動的に表示されます

**ヒント**: 最初は小さいフレーム数（25〜49）で試して、結果を確認してから増やすことをお勧めします。

In [ ]:
# --- 動画生成パラメータの設定 ---

# ポジティブプロンプト: 生成したい動画の内容を詳細に記述
positive_prompt = "Flowers growing from the sides of a vase" # @param {"type":"string"}

# ネガティブプロンプト: 避けたい要素を列挙
negative_prompt = "low quality, worst quality, deformed, distorted, disfigured, motion smear, motion artifacts, fused fingers, bad anatomy, weird hand, ugly" # @param {"type":"string"}

# 動画の解像度（32の倍数である必要があります）
width = 512 # @param {"type":"number"}
height = 768 # @param {"type":"number"}

# ランダムシード: 同じ値で同じ結果を再現できます
seed = 397166166231987 # @param {"type":"integer"}

# 生成ステップ数: 多いほど高品質だが時間がかかる
steps = 25 # @param {"type":"integer", "min":1, "max":100}

# CFGスケール: プロンプトへの忠実度（2.0前後を推奨）
cfg_scale = 2.05 # @param {"type":"number", "min":1, "max":20}

# サンプラー: 生成アルゴリズムの種類
sampler_name = "euler" # @param ["euler", "dpmpp_2m", "ddim", "lms"]

# フレーム数: 多いほど長い動画になる
frames = 49 # @param {"type":"integer", "min":1, "max":120}

# ガイド強度: 終了画像への影響度（0〜1、1が最大）
guide_strength = 1 # @param {"type":"number", "min":0, "max":1}

# ガイドフレーム: 終了画像を配置するフレーム位置（-1で最後のフレーム）
guide_frame = -1 # @param {"type":"integer"}

# --- 動画生成の実行 ---
print("動画生成ワークフローを開始します...")

# 推論モード（学習しない）で動画生成を実行
with torch.inference_mode():
    generate_video(
        image_path=None,  # Noneの場合、アップロードを促す
        guide_image_path=None,  # Noneの場合、アップロードを促す
        positive_prompt=positive_prompt,
        negative_prompt=negative_prompt,
        width=width,
        height=height,
        seed=seed,
        steps=steps,
        cfg_scale=cfg_scale,
        sampler_name=sampler_name,
        length=frames,
        guide_strength=guide_strength,
        guide_frame=guide_frame
    )

# 生成完了後、GPUメモリをクリーンアップ
clear_gpu_memory()